In [1]:
import pandas as pd
from tqdm import tqdm
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap

df = pd.read_csv('/home/maia-user/myl/sc/Datasets/processed_total.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.750561,1.416667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.755108,1.333333e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [2]:
print(df.shape)
print(df["Label"].value_counts())

(2827876, 79)
Label
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [3]:
# 假设你已经加载好 df，并完成归一化，包含 "Label" 列
rare_classes = ["Infiltration", "Web Attack � XSS", "Web Attack � Sql Injection", "Heartbleed"]
# 固定 BENIGN 样本数（防止混淆 GAN 效果）
benign_df = df[df["Label"] == "BENIGN"].sample(n=10000, random_state=42)
all_attack_df = df[df["Label"] != "BENIGN"]
non_rare_attack_df = all_attack_df[~all_attack_df["Label"].isin(rare_classes)]
df_raw = pd.concat([benign_df, all_attack_df], ignore_index=True)

In [4]:
from cgan_module import cgan_augment
from Acgan import acgan_augment
from adaptive_cgan_module import adaptive_cgan_multi_round
from evaluate import evaluate_pipeline, evaluate_on_original_rare_samples

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier

import pandas as pd

# ⚙️ 模型定义
models = {
    "RandomForest": lambda: RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": lambda: XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
    # "LogisticRegression": lambda: LogisticRegression(max_iter=1000),
    # "LightGBM": lambda: LGBMClassifier()
}

# ✅ 提前生成 CGAN / ACGAN（与模型无关，可共用）
df_cgan = cgan_augment(df, rare_classes, generate_per_class=125)
df_acgan = acgan_augment(df, rare_classes, generate_per_class=125)

# ✅ 拿出增强数据
rare_cgan = df_cgan[df_cgan["Label"].isin(rare_classes)]
rare_acgan = df_acgan[df_acgan["Label"].isin(rare_classes)]

# ✅ 汇总结果容器
results_all = []

# ✅ 多模型评估循环
for model_name, model_func in models.items():
    print(f"\n🚀 正在评估模型: {model_name}")

    # ✅ 当前模型使用 Adaptive 增强（按模型 recall 分配）
    df_adaptive = adaptive_cgan_multi_round(
        df, rare_classes,
        generate_per_round=250,
        max_rounds=2,
        recall_threshold=0.90,
        model_func=model_func
)

    rare_adaptive = df_adaptive[df_adaptive["Label"].isin(rare_classes)]

    # ✅ 构建三份增强数据对应的完整训练集
    df_cgan_full = pd.concat([benign_df, non_rare_attack_df, rare_cgan], ignore_index=True)
    df_acgan_full = pd.concat([benign_df,non_rare_attack_df, rare_acgan], ignore_index=True)
    df_adaptive_full = pd.concat([benign_df,non_rare_attack_df, rare_adaptive], ignore_index=True)

    # ✅ A. Baseline
    clf_base, X_base, _, _, le_base = evaluate_pipeline(
        df_raw, f"{model_name} Baseline", rare_classes=rare_classes, model_cls=model_func
    )

    # ✅ B. CGAN
    clf_cgan, X_cgan, _, _, le_cgan = evaluate_pipeline(
        df_cgan_full, f"{model_name} CGAN增强", rare_classes=rare_classes, model_cls=model_func
    )

    # ✅ C. ACGAN
    clf_acgan, X_acgan, _, _, le_acgan = evaluate_pipeline(
        df_acgan_full, f"{model_name} ACGAN增强", rare_classes=rare_classes, model_cls=model_func
    )

    # ✅ D. Adaptive
    clf_adapt, X_adapt, _, _, le_adapt = evaluate_pipeline(
        df_adaptive_full, f"{model_name} Adaptive增强", rare_classes=rare_classes, model_cls=model_func
    )

    # ✅ 原始 rare 样本评估
    for label, clf, le, tag in [
        ("Baseline", clf_base, le_base, "Base"),
        ("CGAN", clf_cgan, le_cgan, "CGAN"),
        ("ACGAN", clf_acgan, le_acgan, "ACGAN"),
        ("Adaptive", clf_adapt, le_adapt, "Adaptive")
    ]:
        rare_detect = evaluate_on_original_rare_samples(clf, df_raw, le)
        rare_detect["Model"] = model_name
        rare_detect["Enhancement"] = tag
        results_all.append(rare_detect)

# ✅ 结果汇总展示
final_df = pd.concat(results_all, ignore_index=True)
pivot_df = final_df.pivot_table(
    index=["Model", "Class", "Original Sample Count"],
    columns="Enhancement",
    values=["Detected Correctly", "Recall"]
).reset_index()

print("\n📊 多模型 + 多增强方法 对原始 Rare 攻击识别对比：")
print(pivot_df.to_string(index=False))



🔍 正在增强类别: Infiltration
[0/500] Loss_D: 1.3876, Loss_G: 0.7181
[100/500] Loss_D: 0.7640, Loss_G: 1.5138
[200/500] Loss_D: 0.2330, Loss_G: 3.3256
[300/500] Loss_D: 0.2162, Loss_G: 3.2107
[400/500] Loss_D: 1.3390, Loss_G: 1.0983
✅ 生成完成: Infiltration → 125 条

🔍 正在增强类别: Web Attack � XSS
[0/500] Loss_D: 1.3812, Loss_G: 0.7089
[100/500] Loss_D: 0.8903, Loss_G: 1.2937
[200/500] Loss_D: 0.2283, Loss_G: 3.0957
[300/500] Loss_D: 0.7518, Loss_G: 1.2144
[400/500] Loss_D: 0.7480, Loss_G: 1.3261
✅ 生成完成: Web Attack � XSS → 125 条

🔍 正在增强类别: Web Attack � Sql Injection
[0/500] Loss_D: 1.3944, Loss_G: 0.6693
[100/500] Loss_D: 0.8365, Loss_G: 1.4892
[200/500] Loss_D: 0.2875, Loss_G: 2.7179
[300/500] Loss_D: 0.4452, Loss_G: 1.9149
[400/500] Loss_D: 1.0199, Loss_G: 1.1861
✅ 生成完成: Web Attack � Sql Injection → 125 条

🔍 正在增强类别: Heartbleed
[0/500] Loss_D: 1.4059, Loss_G: 0.7351
[100/500] Loss_D: 0.6680, Loss_G: 1.8285
[200/500] Loss_D: 0.2555, Loss_G: 2.8657
[300/500] Loss_D: 0.2797, Loss_G: 2.4171
[400/500] Lo

/home/maia-user/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [09:27:46] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 分类报告 ([Adaptive-R1] Baseline):
                            precision    recall  f1-score   support

                    BENIGN     1.0000    1.0000    1.0000    454264
                Heartbleed     0.0000    0.0000    0.0000         2
              Infiltration     1.0000    0.8571    0.9231         7
Web Attack � Sql Injection     1.0000    1.0000    1.0000         4
          Web Attack � XSS     0.9920    0.9466    0.9688       131

                  accuracy                         1.0000    454408
                 macro avg     0.7984    0.7607    0.7784    454408
              weighted avg     1.0000    1.0000    1.0000    454408


📌 Rare 类别测试集表现：
                     Class  Test Samples  Correctly Predicted  Recall
              Infiltration             7                    6  0.8571
          Web Attack � XSS           131                  124  0.9466
Web Attack � Sql Injection             4                    4  1.0000
                Heartbleed             2              

/home/maia-user/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maia-user/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maia-user/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[0/500] Loss_D: 1.3689, Loss_G: 0.7064
[100/500] Loss_D: 1.0470, Loss_G: 1.0621
[200/500] Loss_D: 0.2739, Loss_G: 2.7245
[300/500] Loss_D: 0.4637, Loss_G: 2.4827
[400/500] Loss_D: 1.1302, Loss_G: 1.2036
✅ Infiltration 增强完成 +31
[0/500] Loss_D: 1.4002, Loss_G: 0.7289
[100/500] Loss_D: 0.8415, Loss_G: 1.2472
[200/500] Loss_D: 0.3565, Loss_G: 2.7973
[300/500] Loss_D: 0.1375, Loss_G: 3.4801
[400/500] Loss_D: 0.8879, Loss_G: 1.3797
✅ Heartbleed 增强完成 +218

🔁 Round 2 / 2 --------------------------


...


📊 分类报告 (XGBoost CGAN增强):
                            precision    recall  f1-score   support

                    BENIGN     0.9950    0.9965    0.9958      2000
                       Bot     0.9873    0.9923    0.9898       391
                      DDoS     1.0000    1.0000    1.0000     25605
             DoS GoldenEye     0.9971    0.9961    0.9966      2059
                  DoS Hulk     0.9997    0.9998    0.9997     46025
          DoS Slowhttptest     0.9955    0.9973    0.9964      1100
             DoS slowloris     0.9965    0.9931    0.9948      1159
               FTP-Patator     1.0000    1.0000    1.0000      1587
                Heartbleed     1.0000    1.0000    1.0000        27
              Infiltration     1.0000    0.9375    0.9677        32
                  PortScan     0.9998    0.9998    0.9998     31761
               SSH-Patator     1.0000    1.0000    1.0000      1180
  Web Attack � Brute Force     0.7210    0.8642    0.7861       302
Web Attack � Sql Inje

/home/maia-user/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [09:30:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 分类报告 (XGBoost ACGAN增强):
                            precision    recall  f1-score   support

                    BENIGN     0.9955    0.9975    0.9965      2000
                       Bot     0.9898    0.9923    0.9911       391
                      DDoS     1.0000    0.9999    1.0000     25605
             DoS GoldenEye     0.9981    0.9956    0.9968      2059
                  DoS Hulk     0.9997    0.9998    0.9997     46025
          DoS Slowhttptest     0.9955    0.9973    0.9964      1100
             DoS slowloris     0.9965    0.9931    0.9948      1159
               FTP-Patator     1.0000    1.0000    1.0000      1587
                Heartbleed     1.0000    1.0000    1.0000        27
              Infiltration     1.0000    0.9375    0.9677        32
                  PortScan     0.9998    0.9998    0.9998     31761
               SSH-Patator     1.0000    1.0000    1.0000      1180
  Web Attack � Brute Force     0.7283    0.8874    0.8000       302
Web Attack � Sql Inj

/home/maia-user/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [09:30:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 分类报告 (XGBoost Adaptive增强):
                            precision    recall  f1-score   support

                    BENIGN     0.9955    0.9965    0.9960      2000
                       Bot     0.9848    0.9974    0.9911       391
                      DDoS     1.0000    0.9999    0.9999     25605
             DoS GoldenEye     0.9966    0.9976    0.9971      2059
                  DoS Hulk     0.9997    0.9998    0.9997     46025
          DoS Slowhttptest     0.9973    0.9973    0.9973      1100
             DoS slowloris     0.9974    0.9940    0.9957      1159
               FTP-Patator     1.0000    1.0000    1.0000      1587
                Heartbleed     1.0000    1.0000    1.0000        46
              Infiltration     1.0000    0.8800    0.9362        25
                  PortScan     0.9998    0.9997    0.9998     31761
               SSH-Patator     1.0000    1.0000    1.0000      1180
  Web Attack � Brute Force     0.7216    0.8870    0.7958       301
Web Attack � Sql 